In [1]:
import nltk
import warnings
warnings.filterwarnings("ignore")
from IPython.display import Image 
from IPython.core.display import Image, display
from collections import Counter
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
import pymongo
import random
from itertools import permutations
from nltk.tokenize import sent_tokenize, word_tokenize
from tqdm import tqdm
import os
tqdm.pandas()
from datetime import datetime
import requests

import dropbox
import shutil


# import MisterProper
import UploadDropbox

import spacy
from spacy_langdetect import LanguageDetector
import en_core_web_sm
import re
import itertools
import os
from shutil import rmtree


nlp = en_core_web_sm.load()
nlp.add_pipe(LanguageDetector(), name='language_detector', last=True)

username = "tagger-admin"
password = "tvaiadmin"
db_client = pymongo.MongoClient('104.198.62.226', username=username,
                                password=password,
                                authSource='tags',
                                authMechanism='SCRAM-SHA-256', port=27017).tags

In [2]:
timeout = 10.0
access_token = '7wYL8EDqrSAAAAAAAAAjFKIa8fjneyf3Z6xIHPeK6N09ZyGZrVjY4RZGQPSajNY-'
dbx = dropbox.Dropbox(access_token, timeout=timeout)
dbx.files_list_folder(path="/graphic.one/")

ListFolderResult(entries=[FolderMetadata(name='PYTHON', id='id:FLbIeQ9wdBAAAAAAAABPhg', path_lower='/graphic.one/python', path_display='/Graphic.one/PYTHON', parent_shared_folder_id=None, shared_folder_id=None, sharing_info=None, property_groups=None), FolderMetadata(name='ImageToText', id='id:-Y4ePi3oiZAAAAAAAACywg', path_lower='/graphic.one/imagetotext', path_display='/Graphic.one/ImageToText', parent_shared_folder_id=None, shared_folder_id=None, sharing_info=None, property_groups=None), FolderMetadata(name='Tagging', id='id:-Y4ePi3oiZAAAAAAAACyww', path_lower='/graphic.one/tagging', path_display='/Graphic.one/Tagging', parent_shared_folder_id=None, shared_folder_id=None, sharing_info=None, property_groups=None), FolderMetadata(name='ImageCategorization', id='id:-Y4ePi3oiZAAAAAAAAGadg', path_lower='/graphic.one/imagecategorization', path_display='/Graphic.one/ImageCategorization', parent_shared_folder_id=None, shared_folder_id=None, sharing_info=None, property_groups=None), FolderMet

In [17]:
def getImages(folder):
    processedData = []
    for image in folder:
        processedData.append(image.name)
    return processedData

def getMoreThanCan(folder, dbx):
    images = getImages(folder.entries)
    while True:
        folder = dbx.files_list_folder_continue(folder.cursor)
        images += getImages(folder.entries)
        if not folder.has_more:
            break
    return images

def getFilesLists(categories, dbx, folderPath):
    files = []
    folder = dbx.files_list_folder(path=categories.path_lower)
    if folder.has_more:
        data = getMoreThanCan(folder, dbx)
        files += data
        print(folderPath,": ", categories.name, len(data))
    else:
        data = getImages(folder.entries)
        files += data
        print(folderPath,": ", categories.name, len(data))
    return files

def get1dList(data): return list(itertools.chain.from_iterable(data))
    

def getProccessedData(access_token):
    
    timeout = 10.0
    
    dbx = dropbox.Dropbox(access_token, timeout=timeout)
    
    foldersPaths = {
        "train TXT VOL1" : dbx.files_list_folder(path="/graphic.one/Conveoyr/Trasher/TrashTraining/TXT/VOL_1"),
        "train TXT VOL2" : dbx.files_list_folder(path="/graphic.one/Conveoyr/Trasher/TrashTraining/TXT/VOL_2"),        
        "train IMG" : dbx.files_list_folder(path="/graphic.one/Conveoyr/Trasher/TrashTraining/IMG"),
        "val" :dbx.files_list_folder(path="/graphic.one/Conveoyr/Trasher/TrashValidation/TXT")   
    }
    
    TrashFiles = get1dList([get1dList([getFilesLists(categories, dbx, folderPath) for categories in foldersPaths[folderPath].entries]) for folderPath in foldersPaths])
    TrashFilesID = [i[:-4] for i in TrashFiles]
    
    print("TOTAL IMAGES PROCESSED:",len(TrashFilesID))
    
    return TrashFilesID

In [18]:
access_token = '7wYL8EDqrSAAAAAAAAAjFKIa8fjneyf3Z6xIHPeK6N09ZyGZrVjY4RZGQPSajNY-'
TrashFiles = getProccessedData(access_token)

train TXT VOL1 :  SourceZero 0
train TXT VOL1 :  ShortTerm 2638
train TXT VOL1 :  MarketDesk 1459
train TXT VOL1 :  Trash 4135
train TXT VOL1 :  FearGreed 151
train TXT VOL2 :  MarketDesk 120
train TXT VOL2 :  ShortTerm 50
train TXT VOL2 :  Trash 7066
train TXT VOL2 :  FearGreed 14
train TXT VOL2 :  SourceZero 1999
train TXT VOL2 :  WithSource 1604
train IMG :  TechnicalChart 3697
train IMG :  BloombergTerminal 676
train IMG :  Picture 1534
train IMG :  ScreenShot 570
val :  NoneEnglish 94
TOTAL IMAGES PROCESSED: 25807


In [83]:
def getDeleted(TrashFiles):
    deletedData = []
    db = db_client[ "tweets_pipeline_v2"].find({"$or": [{"status":"deleted"}, {"status":"deleted_from_analytics"}],
                                                  "extracted_text": {"$exists": True},
                                                  'pipelined_ts': {'$gte': datetime(2020,2,1)}                                                                                               
                                                   })
    for i in tqdm(db, total = db.count()):
        _id = str(i['_id'])
        urls = i["img_urls"]
        if "GutenTagTrash" in i and (i["GutenTagTrash"] == ['G1_Standart'] or i["GutenTagTrash"] == []):
            reason = ['trash']
            deletedData.append([_id, urls, reason])                
        if "GutenTagTrash" not in i:
            reason = ['trash']
            deletedData.append([_id, urls, reason])
    
    totalDeleted = pd.DataFrame(deletedData, columns=["ID", "url", "GutenTagTrash"])    
    totalDeleted = totalDeleted[~totalDeleted.ID.isin(TrashFiles)]
    totalDeleted.reset_index(inplace=True, drop=True)

    return totalDeleted

# NEW DATA FROM MONGO

In [84]:
totalDeleted = getDeleted(TrashFiles)
print("totalDeleted:", totalDeleted.shape[0])
totalDeleted.head(15)

100%|██████████| 19509/19509 [00:02<00:00, 8047.73it/s]

totalDeleted: 4552


,ID,url,GutenTagTrash
0,5e3eb8e2e9642c8c5c4f879d,https://pbs.twimg.com/media/EQLJYiEXUAA-5ki.jpg,[trash]
1,5e3eb91be9642c8c5c4f879e,https://pbs.twimg.com/media/EQLQYrlXsAAZz_k.jpg,[trash]
2,5e4024d6f1f1b24a4dccc90b,https://s.yimg.com/ny/api/res/1.2/tdGZLWU6JSEA...,[trash]
3,5e412a0f7f6d48194fdfc1e2,https://pbs.twimg.com/media/EP8NKd9WkAEO696.jpg,[trash]
4,5e412aac78f4be3478ab8743,https://pbs.twimg.com/media/EOaUjeSUYAAhuQB.jpg,[trash]
5,5e412b179884ef77881c92e1,https://pbs.twimg.com/media/EQL7U9WUwAADa9Z.jpg,[trash]
6,5e412b1a0520e07a588f6264,https://pbs.twimg.com/media/EQBGRxfUcAoLaon.png,[trash]
7,5e412b50b518bd8eb23692d5,https://pbs.twimg.com/media/ELdmvPZVUAEHLp7.jpg,[trash]
8,5e412b557769992dcbca84e2,https://pbs.twimg.com/media/EPKre4UUUAAwB4x.jpg,[trash]
9,5e412b582ee5dee139a3990f,https://pbs.twimg.com/media/ENFLIhTUcAEvig0.jpg,[trash]


# DOWNLOAD NEW DATA

In [85]:
def download(folderName,_id, extension,r, MainDir):
    if extension in [".jpg", ".png"]:
        if not os.path.exists(MainDir+"/"+folderName):
            os.makedirs(MainDir+"/"+folderName)
        path = MainDir+folderName+"/"+str(_id) + extension
#         print(path)
        with open(path, 'wb') as f:
            f.write(r.content)

In [89]:
def FolderPreparator(x, MainDir):
    
    if not os.path.exists(MainDir):
        os.mkdir(MainDir)    
        
    folders = x[x==True].index.values
    _id, url, reason = x["ID"], x["url"], x["GutenTagTrash"]
    extension = url[-4:]
    try:
        r = requests.get(url)
        if len(reason) > 0:
            for reason_folder in reason:
    #             print(reason_folder)
                download(reason_folder,_id, extension, r, MainDir)
        else:
            download("other",_id, extension, r, MainDir)
    except Exception as err:
        pass

# SEND TEST FILES TO DROPBOX

In [90]:
mainDir = "newData/"

In [91]:
totalDeleted = totalDeleted.progress_apply(lambda x: FolderPreparator(x, mainDir), axis=1)

100%|██████████| 4552/4552 [08:33<00:00,  8.87it/s] 


In [92]:
!zip -r deleted14052020.zip newData

updating: newData/ (stored 0%)
updating: newData/trash/ (stored 0%)
updating: newData/trash/5e9499707f97bd7ea0afad83.jpg (deflated 13%)
updating: newData/trash/5eaf933768c79391323a0f61.jpg (deflated 4%)
updating: newData/trash/5ead417719945cd370e8f17a.jpg (deflated 1%)
updating: newData/trash/5e94848d2ec9f38b4889471e.jpg (deflated 9%)
updating: newData/trash/5e95c7b3c40ca5cfa9bb5b23.png (deflated 2%)
updating: newData/trash/5ead77a5330906471b1b381d.png (deflated 2%)
updating: newData/trash/5eafabea54d777912a0f1cd6.png (deflated 1%)
updating: newData/trash/5e8ccec2c67bc7c950599fa3.jpg (deflated 14%)
updating: newData/trash/5eb9df3dba20e9bec47ab4f4.jpg (deflated 15%)
updating: newData/trash/5eac63044bd4180f4123653d.jpg (deflated 20%)
updating: newData/trash/5eb9a75b2c820bffdccef5e7.jpg (deflated 10%)
updating: newData/trash/5e8cc47bfcaeea0901632988.png (deflated 7%)
updating: newData/trash/5eae9ac02966ebb7364eedfe.png (deflated 2%)
updating: newData/trash/5e8dcb5edced32ea4f763ef1.jpg (de

updating: newData/trash/5e8ec16e8ab33c6789111a18.jpg (deflated 15%)
updating: newData/trash/5ead8f48202090a5be088c40.png (deflated 20%)
updating: newData/trash/5eaeb9e08a88ebe965bb848c.jpg (deflated 5%)
updating: newData/trash/5eb89a178fc8a18789a28e13.jpg (deflated 23%)
updating: newData/trash/5eadc20957ec210b8789db2f.jpg (deflated 19%)
updating: newData/trash/5eaf4b1eafd4735023f81ad0.jpg (deflated 8%)
updating: newData/trash/5e8e4e86e4b520a3bd1d2bca.jpg (stored 0%)
updating: newData/trash/5eb82eaaded5a2c7bf544075.jpg (deflated 5%)
updating: newData/trash/5e91b7a57ba36c2a34096355.jpg (deflated 33%)
updating: newData/trash/5e917415e5105c0ec9c0aec0.jpg (deflated 11%)
updating: newData/trash/5eaabcf9690ce23d107a2ac8.jpg (deflated 13%)
updating: newData/trash/5eabd1d078c8e4e7f746c3d4.jpg (deflated 12%)
updating: newData/trash/5e5faa6669cae70230fe5922.png (deflated 2%)
updating: newData/trash/5e61c2b11abb15638d1a79cb.jpg (deflated 3%)
updating: newData/trash/5e8d334831f47f045dfee109.jpg (de

updating: newData/trash/5e8e7e70277930b7a36a67b7.jpg (deflated 10%)
updating: newData/trash/5eaf7351b50f86d292d69ba7.jpg (deflated 20%)
updating: newData/trash/5e8ee3c77eb32e485f15d5d2.jpg (deflated 11%)
updating: newData/trash/5e91c553a9e76128dc8c4177.jpg (deflated 21%)
updating: newData/trash/5e904d746640676462d73e3e.png (deflated 11%)
updating: newData/trash/5e8e737f7eb32e485f15d5a4.jpg (deflated 4%)
updating: newData/trash/5e965a37f76c281df0c17031.jpg (deflated 22%)
updating: newData/trash/5eb9df4802678cf1a5852a87.png (deflated 3%)
updating: newData/trash/5e91dad57b24ce8737301bae.png (deflated 0%)
updating: newData/trash/5eb9b5faf1bacf997c05f681.jpg (deflated 3%)
updating: newData/trash/5eb9b3d1c4b4676562c4332f.png (deflated 0%)
updating: newData/trash/5eae3a9a05d6dcebc475b328.jpg (deflated 20%)
updating: newData/trash/5e605cda07d2c98edaf0cfb3.jpg (deflated 3%)
updating: newData/trash/5e94949018e6ded480820af9.jpg (deflated 21%)
updating: newData/trash/5e95e5070d27644bde05a551.jpg (

updating: newData/trash/5e9498e97f97bd7ea0afad82.png (deflated 1%)
updating: newData/trash/5eb8b79c35232baa30b3164f.png (deflated 1%)
updating: newData/trash/5eaae433bdff4f14b3fd8c79.jpg (deflated 2%)
updating: newData/trash/5e8f1c9b60fe0fa526fae0fb.jpg (deflated 18%)
updating: newData/trash/5e9648dfa66fdb24430fe970.png (deflated 3%)
updating: newData/trash/5e959bf70342c3a50f47a2ac.jpg (deflated 9%)
updating: newData/trash/5e94e3bf0173a631daf288c0.png (deflated 1%)
updating: newData/trash/5eb9a987086e400775d3d70c.jpg (deflated 8%)
updating: newData/trash/5eadc3724d8f080e7d676adb.jpg (deflated 14%)
updating: newData/trash/5eba0d2db64c5f3ad640593a.jpg (deflated 19%)
updating: newData/trash/5eaf883ec4a16d840a4dece1.jpg (deflated 5%)
updating: newData/trash/5eaabde24d6798fd0f358247.jpg (deflated 9%)
updating: newData/trash/5eabdf96ce2cb43abdd51970.jpg (deflated 9%)
updating: newData/trash/5e8e73681ca32cf3d6697faa.jpg (deflated 5%)
updating: newData/trash/5e8fda45b80d9e2b9daef0d4.jpg (defla

updating: newData/trash/5e8f10844a28055d461ea1bc.png (deflated 5%)
updating: newData/trash/5e90d975fec218434d7d573e.jpg (deflated 21%)
updating: newData/trash/5e95b22b653ece2c5a04d4a1.jpg (deflated 11%)
updating: newData/trash/5eac7ef5241be8467236b45a.png (deflated 3%)
updating: newData/trash/5eb89e3618346cfa3f9b88dd.png (deflated 17%)
updating: newData/trash/5eae2f07e8689ae4c5579f03.jpg (deflated 14%)
updating: newData/trash/5e94cdb39a23220124984624.jpg (deflated 11%)
updating: newData/trash/5ead29d958433b464c7d443c.jpg (deflated 3%)
updating: newData/trash/5eb97ec45cf4633cb71b8275.jpg (deflated 8%)
updating: newData/trash/5eb8b7a2e36743c0aa84141d.png (deflated 6%)
updating: newData/trash/5e8dd204b2ff10890d91c0b4.png (deflated 11%)
updating: newData/trash/5eabadc1e6bd4afae1fd3385.png (deflated 3%)
updating: newData/trash/5e91ac1db0702c1de6506dee.jpg (deflated 15%)
updating: newData/trash/5e8dccd2f9344d11216a672e.jpg (deflated 17%)
updating: newData/trash/5e8df1ac6af5044f29f2e5ca.png (

...

  adding: newData/trash/5e413d2140b4b12b0b7d3fcf.jpg (deflated 1%)
  adding: newData/trash/5e90e2b7f9c86240eba21791.jpg (deflated 2%)
  adding: newData/trash/5e413341023ab1efb412bad6.jpg (deflated 11%)
  adding: newData/trash/5e415d398c81ee1b0ea8bb8e.jpg (deflated 12%)
  adding: newData/trash/5e4164c1c69a292b20023d34.jpg (deflated 7%)
  adding: newData/trash/5e41579fcf0291b2f06d5e5d.jpg (deflated 0%)
  adding: newData/trash/5e416288f8a018d22b22cd32.png (deflated 4%)
  adding: newData/trash/5e416262023ab1efb412bafc.jpg (deflated 1%)
  adding: newData/trash/5e41455a1ce524ba77e17bae.jpg (deflated 7%)
  adding: newData/trash/5e415b30794c0d03899ac1f7.jpg (deflated 4%)
  adding: newData/trash/5e41468fd869508f1a03e4d3.png (deflated 2%)
  adding: newData/trash/5e412eb1197eeb0287dad4ff.jpg (stored 0%)
  adding: newData/trash/5e41322f6ac499509110e4d3.jpg (deflated 26%)
  adding: newData/trash/5e4168bcce3eb687f260ed01.png (deflated 3%)
  adding: newData/trash/5e413bd30520e07a588f6286.jpg (deflate

  adding: newData/trash/5e41447f7daedbbf4f063de2.jpg (deflated 1%)
  adding: newData/trash/5e4140200520e07a588f628a.jpg (deflated 11%)
  adding: newData/trash/5e6020e2c6c77def60df82de.png (deflated 2%)
  adding: newData/trash/5e413c8f7769992dcbca84fb.jpg (deflated 8%)
  adding: newData/trash/5e413152bc8f8b0d73d97f08.jpg (deflated 4%)
  adding: newData/trash/5e8dfd407accea94c90daa2a.png (deflated 1%)
  adding: newData/trash/5e413148ce3eb687f260ecc5.jpg (deflated 22%)
  adding: newData/trash/5e4165a9bd7405c22f28fa72.jpg (deflated 28%)
  adding: newData/trash/5e415e634b0830a7e55af980.png (deflated 1%)
  adding: newData/trash/5e4164ab3b1a37b27bb5adbc.jpg (deflated 5%)
  adding: newData/trash/5e41366ee340eda2f77d55f8.jpg (deflated 2%)
  adding: newData/trash/5e413389bd3651614360be60.png (deflated 2%)
  adding: newData/trash/5e90a5a10771bf169dd66e83.jpg (deflated 13%)
  adding: newData/trash/5e41471066377924c8dd9f1a.jpg (deflated 5%)
  adding: newData/trash/5e6029a72b78c8f62d48b6fc.jpg (defl

  adding: newData/trash/5e41644ee712a66b81719565.jpg (deflated 1%)
  adding: newData/trash/5e8db7774a8150b29a2fbcd4.jpg (deflated 17%)
  adding: newData/trash/5e41651f40b4b12b0b7d3ff8.jpg (deflated 3%)
  adding: newData/trash/5e6022a70eaf10230fa6b6a9.jpg (deflated 4%)
  adding: newData/trash/5e414b81c69a292b20023d19.jpg (deflated 3%)
  adding: newData/trash/5e413eae8d98474fbe953980.jpg (deflated 0%)
  adding: newData/trash/5e413a1d05098cd33b2e920b.png (deflated 9%)
  adding: newData/trash/5e412fe28c81ee1b0ea8bb6a.jpg (deflated 9%)
  adding: newData/trash/5e4165081ce524ba77e17bca.jpg (deflated 0%)
  adding: newData/trash/5ebbffa139d12c189475cd0f.png (deflated 5%)
  adding: newData/trash/5e413ef7ccf3f921ae0e546d.jpg (deflated 5%)
  adding: newData/trash/5e414e54ccf3f921ae0e547e.png (deflated 1%)
  adding: newData/trash/5e414b217a607a7cea895142.jpg (deflated 5%)
  adding: newData/trash/5ebbfcfcf7d67bbe5f8778e8.png (deflated 5%)
  adding: newData/trash/5e412fc89c46392036296adc.jpg (deflate

  adding: newData/trash/5e415a3f29aa06f51424e319.jpg (deflated 1%)
  adding: newData/trash/5e414220236cd951f58c1089.jpg (deflated 7%)
  adding: newData/trash/5e4155c8ccf3f921ae0e5489.jpg (deflated 24%)
  adding: newData/trash/5e4144fbaeadbea3f397c04e.png (deflated 8%)
  adding: newData/trash/5e416589e188f348a5309ee2.jpg (deflated 3%)
  adding: newData/trash/5e41415f514f1535fc414304.jpg (deflated 4%)
  adding: newData/trash/5e413ade0520e07a588f6285.jpg (deflated 31%)
  adding: newData/trash/5e4154cb9c46392036296b00.jpg (deflated 7%)
  adding: newData/trash/5e4132d7e88c54dbe67f773a.jpg (deflated 3%)
  adding: newData/trash/5e4137c729aa06f51424e2fb.jpg (deflated 15%)
  adding: newData/trash/5e414b3222e4430f53ef1b4b.jpg (deflated 12%)
  adding: newData/trash/5e414293e712a66b81719542.jpg (deflated 10%)
  adding: newData/trash/5e8ca9627f1217b835cdd677.jpg (deflated 0%)
  adding: newData/trash/5e41468e17924e76e58edbe1.png (deflated 2%)
  adding: newData/trash/5e5fe7bd89a7f7e2363bdafa.png (def

  adding: newData/trash/5e412f2105098cd33b2e9204.png (deflated 0%)
  adding: newData/trash/5e415555ccf3f921ae0e5487.png (deflated 13%)
  adding: newData/trash/5ebbffab61127506a55961de.png (deflated 21%)
  adding: newData/trash/5e41311ee712a66b81719516.jpg (deflated 2%)
  adding: newData/trash/5e4154e4e712a66b81719559.png (deflated 2%)
  adding: newData/trash/5e41546b21233b84f9cec9a8.jpg (deflated 11%)
  adding: newData/trash/5e413791eb8f57d996223f59.jpg (deflated 18%)
  adding: newData/trash/5e414681f8a018d22b22cd13.jpg (deflated 5%)
  adding: newData/trash/5e4161d021233b84f9cec9b8.jpg (deflated 9%)
  adding: newData/trash/5ebc21446b781a2657dd9223.png (deflated 5%)
  adding: newData/trash/5e41487397c38359a4024747.jpg (deflated 10%)
  adding: newData/trash/5e605453702f0b9285706714.jpg (deflated 1%)
  adding: newData/trash/5e415a94c69a292b20023d27.png (deflated 2%)
  adding: newData/trash/5e4145bace3eb687f260ecdc.png (deflated 0%)
  adding: newData/trash/5e4147039884ef77881c92fa.jpg (def

In [93]:
UploadDropbox.upload("deleted14052020.zip", "/graphic.one/Conveoyr/deleted14052020.zip")

Uploaded: 482344960it [02:40, 2996335.26it/s]                               

FileMetadata(name='deleted14052020.zip', id='id:-Y4ePi3oiZAAAAAAAAR1jw', client_modified=datetime.datetime(2020, 5, 14, 16, 24, 58), server_modified=datetime.datetime(2020, 5, 14, 16, 24, 58), rev='015a59e234758c200000001965752c0', size=479135051, path_lower='/graphic.one/conveoyr/deleted14052020.zip', path_display='/Graphic.one/Conveoyr/deleted14052020.zip', parent_shared_folder_id='6817272512', media_info=None, symlink_info=None, sharing_info=FileSharingInfo(read_only=False, parent_shared_folder_id='6817272512', modified_by='dbid:AABmCuXAyItIx-2z4P3ETEKOHjI7RrF84HM'), is_downloadable=True, export_info=None, property_groups=None, has_explicit_shared_members=None, content_hash='2d5f3914799568038edc8483bee48452f6b297b7695f062eb3b96d1004fe1ad4')


In [11]:
targetDir = "/graphic.one/Conveoyr/TrashValidation/TXT"

counter = 0
for folder in os.listdir(mainDir):
    print(folder)
    subPath = mainDir+folder
    print(subPath)
    for file in tqdm(os.listdir(subPath), total = len(os.listdir(subPath)), desc=folder):
        if file[:-4] not in TrashFiles:
            counter+=1
            file_path = subPath+"/"+file
            target_path = targetDir+"/"+folder+"/"+file
            try:
                UploadDropbox.uploadImage(file_path, target_path)
            except Exception as error:
                print(error)
                
# if os.path.exists(mainDir) == True:
#     shutil.rmtree(mainDir)                
# print(counter)

G1_Standart:   0%|          | 0/17 [00:00<?, ?it/s]

G1_Standart
newData/G1_Standart


KeyboardInterrupt: 

In [ ]:
# def DownloadDeleted(args):
    

In [28]:
args = {
    "statuses": [{"status":"deleted"}, {"status":"deleted_from_analytics"}],
}

In [29]:
def getDeleted(TrashFiles, args):
    deletedData = []
    db = db_client[ "tweets_pipeline_v2"].find({"$or": args["statuses"],
                                              "extracted_text": {"$exists": True},
#                                                'tweet_timestamp': {'$gte': datetime(2020,1,1)}                                                                                               
                                                   })
    
    for i in tqdm(db, total = db.count()):
        _id = str(i['_id'])
        urls = i["img_urls"]
        reason = []
        if "GutenTagTrash" in i:
            reason = i["GutenTagTrash"]        
            if 'G1_Standart' in reason:
                deletedData.append([_id, urls, reason])
                
    
    totalDeleted = pd.DataFrame(deletedData, columns=["ID", "url", "GutenTagTrash"])    
    totalDeleted = totalDeleted[~totalDeleted.ID.isin(TrashFiles)]
    totalDeleted.reset_index(inplace=True, drop=True)

    return totalDeleted

getDeleted(TrashFiles, args)

100%|██████████| 47357/47357 [00:04<00:00, 9968.30it/s]


,ID,url,GutenTagTrash
0,5dd127d9edee5484619cc07b,http://pbs.twimg.com/media/EAZ7jeiXkAI6SkC.png,[G1_Standart]
1,5ddaaf6386facb54ae6c5cf2,http://pbs.twimg.com/media/D34CWPAXsAArZUu.jpg,"[G1_Standart, Trash]"
2,5defe4e71f5e287ece65eb0c,http://pbs.twimg.com/media/DnCjK_qU8AEEE1X.jpg,"[G1_Standart, Trash]"
3,5dfb6546131cdf9d830ab6b2,http://pbs.twimg.com/media/EALP6kIWsAA8UOG.png,[G1_Standart]
4,5dfb748e90b37c797d481d6d,http://pbs.twimg.com/media/CWbeWW9U4AAGp_x.png,"[G1_Standart, Trash]"
5,5dfb9a1320afaad67ff25929,http://pbs.twimg.com/media/D9_KcJ3XkAA0-n4.jpg,"[G1_Standart, Trash]"
6,5dfba9bc521d05797e7f6a75,http://pbs.twimg.com/media/C40VkjTUYAAxskO.jpg,"[G1_Standart, Trash]"
7,5dfbb510890c58587407dc8e,http://pbs.twimg.com/media/D2704kuXcAAktVv.png,[G1_Standart]
8,5dfbd0206e02145be62731f0,http://pbs.twimg.com/media/DZurkYuX4AAHo5h.jpg,[G1_Standart]
9,5dfbdbcc521d05797e7f7900,http://pbs.twimg.com/media/D6W70M4X4AIPYor.png,[G1_Standart]
